## Pull data out of zipped folders into one location

In [4]:
#Commented out because it is done
'''
zip_directory = 'data/WedgeZipOfZips'
output_directory = 'data/Wedge_full'  # The folder where you want to store the extracted files

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Loop through each zip file in the directory
for zip_file in os.listdir(zip_directory):
    if zip_file.endswith('.zip'):  # Check if it's a zip file
        zip_file_path = os.path.join(zip_directory, zip_file)

        # Open the zip file
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            # Extract all contents into the output folder
            zip_ref.extractall(output_directory)
'''

## Task 2

Setup BigQuery

In [49]:
from google.cloud import bigquery
import polars as pl


In [50]:
client = bigquery.Client(project="rhamre")


c:\Users\rache\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Create table with all unique card numbers, randomize for sample, find all transactions from those card numbers

In [71]:
query = '''
WITH unique_card_nos AS (
    SELECT DISTINCT card_no
    FROM `rhamre.wedge.transArchive*`
    WHERE card_no != 3
    ORDER BY RAND()
    LIMIT 450
)

SELECT *
FROM `rhamre.wedge.transArchive*`
WHERE card_no IN (SELECT card_no FROM unique_card_nos)

'''

In [72]:
query_job = client.query(query)
results = query_job.result()

In [73]:
results_df = results.to_dataframe()

c:\Users\rache\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [74]:
results_df.head()

,datetime,register_no,emp_no,trans_no,upc,description,trans_type,trans_subtype,trans_status,department,...,batchHeaderID,local,organic,display,receipt,card_no,store,branch,match_id,trans_id
0,2014-12-16 17:29:05+00:00,9.0,51.0,33.0,17.16DP2,PRODUCE,D,,R,2.0,...,nan,0.0,NaN,nan,0.0,48453.0,1.0,0.0,0.0,11.0
1,2014-12-19 17:57:46+00:00,6.0,12.0,19.0,0000000000044,Old World Rye Bread,I,,,17.0,...,nan,0.0,0.0,nan,0.0,48453.0,1.0,0.0,0.0,5.0
2,2014-10-03 15:33:44+00:00,7.0,24.0,32.0,0000000000045,O.Whole Seed Boule,I,,,17.0,...,nan,0.0,1.0,nan,0.0,48453.0,1.0,0.0,0.0,2.0
3,2014-11-16 14:21:20+00:00,7.0,4.0,39.0,0000000000048,Whole Wheat Batard,I,,,17.0,...,nan,0.0,0.0,nan,0.0,48453.0,1.0,0.0,0.0,9.0
4,2014-10-03 15:33:40+00:00,7.0,24.0,32.0,0000000000054,Crown Challah,I,,,17.0,...,nan,0.0,0.0,nan,0.0,48453.0,1.0,0.0,0.0,1.0


In [75]:
results_df.to_csv("Task_2_Owners_Sample.txt", sep="\t", index=False)